In [3]:
import os
import glob
import codecs
import csv
import pandas as pd
import logging
import math
import numpy as np
from tqdm import tqdm, tqdm_notebook, trange
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from tensorboardX import SummaryWriter
from transformers import (BertConfig, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup)

In [24]:
train_articles = "C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/train"
dev_articles = "C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/dev"
train_SI_labels = "C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/train_SI_labels"
dev_SI_labels = "C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/dev_SI_labels"

device = torch.device("cuda")
n_gpu = torch.cuda.device_count()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("LOG")
MODEL_CLASSES = {"bert": (BertConfig, BertForSequenceClassification, BertTokenizer)}
args = {"data_dir": "datasets/upsampling/",
        "model_type": "bert",
        "model_name": "google-bert/bert-base-multilingual-uncased",
        "output_dir": "datasets/upsampling/output_model_SI",
        "max_seq_length": 128,
        "train_batch_size": 8,
        "eval_batch_size": 8,
        "num_train_epochs": 1,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "gradient_accumulation_steps": 1,
        "logging_steps": 50,
        "save_steps": 2000,
        "overwrite_output_dir": False}

In [25]:
def merge_overlapping(indices_list):
  """
  Merges overlapping indices and sorts indices from list of tuples
  """
  # If no propaganda, return empty list
  if indices_list == []:
    return []

  # Sort the list
  indices_list = sorted(indices_list)
  i = 0

  # Going through tuples from the beginning, see if it overlaps with the nex one 
  # and merge it if so
  while True:
    if i == len(indices_list)- 1: ###!!!! - 1
      break
    
    # If the next one is within the range of current, just delete next one. If
    # overlapping, then merge the range
    elif indices_list[i][1] >= indices_list[i + 1][0]:
      if indices_list[i][1] >= indices_list[i + 1][1]:
        pass
      else:
        indices_list[i] = (indices_list[i][0], indices_list[i+1][1])
      indices_list.pop(i+1)

    # Go to next element if not overlapping
    else:
      i += 1

  return indices_list


In [26]:
def article_to_sequences(article_id, article, tokenizer):
  """
  Divides article into sequences, dividing first by sentences then to powersets
  of the sentences
  """
  # Split the lines by sentences
  curr = 0
  lines = article.split("\n")
  sequences = []
  seq_starts = []
  seq_ends = []

  # For each lines, do:
  for line in lines:
    # If an empty line, just continue
    if line == "":
      curr += 1
      continue

    # Tokenize the line
    tokenized = tokenizer.tokenize(line)

    # For each token, do:
    seq_start = 0
    for ind, token in enumerate(tokenized):
      # Get the token without ## sign
      mod_start_token = token.replace("##", "")

      # Find the start of the sequence in line
      seq_start = line.lower().find(mod_start_token, seq_start)

      # Update the end of the sequence
      seq_end = seq_start

      # For each following tokens in the line, do
      for iter in range(1, len(tokenized) + 1 - ind):
        # Also modify this token
        mod_end_token = tokenized[ind + iter - 1].replace("##", "")
        # Find the end of the token
        seq_end = line.lower().find(mod_end_token, seq_end) + len(mod_end_token)

        sequences.append(tokenizer.convert_tokens_to_string(tokenized[ind: ind + iter]))
        seq_starts.append(curr + seq_start)
        seq_ends.append(curr + seq_end)

      # Update the start of the sequence
      seq_start += len(mod_start_token)

    # Update the current whereabouts
    curr += len(line) + 1

  dataframe = pd.DataFrame(None, range(len(sequences)), ["id", "seq_starts", "seq_ends", "label", "text"])
  dataframe["id"] = [article_id] * len(sequences)
  dataframe["seq_starts"] = seq_starts
  dataframe["seq_ends"] = seq_ends
  dataframe["label"] = [0] * len(sequences)
  dataframe["text"] = sequences
  return dataframe


In [27]:
def article_labels_to_sequences(article, indices_list):
  """
  Divides article into sequences, where each are tagged to be propaganda or not
  """
  # Start at 0 indices, and split the article into lines
  curr = 0
  lines = article.split("\n")
  sequences = {}

  # For each lines, do:
  for line in lines:
    # If an empty line, just continue after adding \n character
    if line == "":
      curr += 1
      continue

    # If nothing in indices_list or current line is not part of propaganda, 
    # just mark it to be none 
    elif indices_list == [] or curr + len(line) <= indices_list[0][0]:
      sequences[line] = 0

    # If current line is part of propaganda, do:
    else:
      # If the propaganda is contained within the line, add it accordingly
      # and pop that indices range
      if curr + len(line) >= indices_list[0][1]:
        sequences[line[:indices_list[0][0] - curr]] = 0
        sequences[line[indices_list[0][0] - curr:indices_list[0][1] - curr]] = 1
        sequences[line[indices_list[0][1] - curr:]] = 0
        indices_list.pop(0)
      # If the propaganda goes over to the next line, add accordingly and 
      # modify that indices range
      else:
        sequences[line[:indices_list[0][0] - curr]] = 0
        sequences[line[indices_list[0][0] - curr:]] = 1
        #indices_list[0][0] = curr + len(line) + 2   #!!!!!!!!!!!!!!!!
        indices_list[0] = (curr + len(line) + 2, indices_list[0][1])  #!!!!
        indices_list[0] = tuple(indices_list[0])                        #!!!
    # Add the current line length plus \n character
    curr += len(line) + 1

  dataframe = pd.DataFrame(None, range(len(sequences)), ["label", "text"])
  dataframe["label"] = sequences.values()  #!!!!!!!!!!!!
  dataframe["text"] = sequences.keys()
  return dataframe

In [28]:
def articles_to_dataframe(article_folder, label_folder):
  """
  Preprocesses the articles into dataframes with sequences with binary tags
  """
  # First sort the filenames and make sure we have label file for each articles
  article_filenames = sorted(glob.glob(os.path.join(article_folder, "*.txt")))
  label_filenames = sorted(glob.glob(os.path.join(label_folder, "*.labels.tsv"))) #!!! format
  assert len(article_filenames) == len(label_filenames)

  # Initialize sequences
  sequences = []

  # For each article, do:
  for i in range(len(article_filenames)):
    # Get the id name
    article_id = os.path.basename(article_filenames[i]).split(".")[0][7:]

    # Read in the article
    with codecs.open(article_filenames[i], "r", encoding="utf8") as f:
      article = f.read()

    # Read in the label file and store indices for SI task

    with open(label_filenames[i], "r") as f:
      reader = csv.reader(f, delimiter="\t")
      indices_list = []
      for row in reader:
        indices_list.append([int(row[1]), int(row[2])])

      # Merge the indices if overlapping
      indices_list = merge_overlapping(indices_list)

    # Add to the sequences
    sequences.append(article_labels_to_sequences(article, indices_list))

  # Concatenate all dataframes
  dataframe = pd.concat(sequences, ignore_index=True)

  return dataframe

In [29]:
def convert_dataframe_to_features(dataframe, max_seq_length, tokenizer):
  """
  Converts dataframe into features dataframe, where each feature will
  take form of [CLS] + A + [SEP]
  """
  # Create features
  features = pd.DataFrame(None, range(dataframe.shape[0]), 
                              ["input_ids", "input_mask", "segment_ids", "label_ids"])

  # For each sequence, do:
  for i in range(len(dataframe)):
    # Set first and second part of the sequences
    tokens = tokenizer.tokenize(dataframe["text"][i])

    # If length of the sequence is greater than max sequence length, truncate it
    if len(tokens) > max_seq_length - 2:
        tokens = tokens[:(max_seq_length - 2)]

    # Concatenate the tokens
    tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]

    # Compute the ids
    segment_ids = [0] * len(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    pad_token = tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0]
    input_ids = input_ids + [pad_token] * padding_length
    input_mask = input_mask + [0] * padding_length
    segment_ids = segment_ids + [0] * padding_length
    label_id = dataframe["label"][i]

    # Assert to make sure we have same length
    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    # Put the data into features dataframe
    features["input_ids"][i] = input_ids
    features["input_mask"][i] = input_mask
    features["segment_ids"][i] = segment_ids
    features["label_ids"][i] = label_id   ###!!!

  return features

In [30]:
def generate_training_dataset_from_articles(articles_folders, labels_folders, tokenizer):
  """
  Generates dataset to go into BERT from articles and labels
  """
  # If generating dataset for evaluation, do:
  logger.info("Generating training dataset...")
    
  # For each articles and labels folder set, turn them into dataframes
  dataframe_list = []
  for i in range(len(articles_folders)):
    logger.info("Generating dataframe for folder %s", articles_folders[i])
    dataframe_list.append(articles_to_dataframe(articles_folders[i], labels_folders[i]))

  # Concatenate the dataframes to make a total dataframe
  dataframe = pd.concat(dataframe_list, ignore_index=True)

  print(dataframe)
  print(dataframe.shape)

  # Process into features dataframe
  logger.info("Creating features from dataframe")
  features = convert_dataframe_to_features(dataframe, args['max_seq_length'], tokenizer)
      
  # Creating TensorDataset from features
  logger.info("Creating TensorDataset from features dataframe")
  all_input_ids = torch.tensor(features["input_ids"], dtype=torch.long)
  all_input_mask = torch.tensor(features["input_mask"], dtype=torch.long)
  all_segment_ids = torch.tensor(features["segment_ids"], dtype=torch.long)
  all_label_ids = torch.tensor(features["label_ids"], dtype=torch.long)

  dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
  return dataset

In [31]:
def train(train_dataset, model, tokenizer):
  """
  Trains the model with training dataset
  """
  # Initialize various necessary objects
  tb_writer = SummaryWriter()
  train_sampler = RandomSampler(train_dataset)
  train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])
    
  # Compute the total time
  t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['num_train_epochs']
  
  # Set the grouped parameters for optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
  
  # Compute warmup steps
  warmup_steps = math.ceil(t_total * args['warmup_ratio'])
  args['warmup_steps'] = warmup_steps if args['warmup_steps'] == 0 else args['warmup_steps']
  
  # Initialize optimizer as Adam with constant weight decay and a linear scheduler with warmup
  optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=args['adam_epsilon'])
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args['warmup_steps'], num_training_steps=t_total)

  # Initialize variables for training
  global_step = 0
  tr_loss, logging_loss = 0.0, 0.0
  model.zero_grad()
  train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
  
  # Start training!
  for _ in train_iterator:
    epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
      model.train()
      batch = tuple(t.to(device) for t in batch)
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2], 
                'labels':         batch[3]}
      outputs = model(**inputs)
      loss = outputs[0]

      if args['gradient_accumulation_steps'] > 1:
        loss = loss / args['gradient_accumulation_steps']
          
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])

      tr_loss += loss.item()
      if (step + 1) % args['gradient_accumulation_steps'] == 0:
        optimizer.step()
        scheduler.step() 
        model.zero_grad()
        global_step += 1

        if args['logging_steps'] > 0 and global_step % args['logging_steps'] == 0:
          tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
          tb_writer.add_scalar('loss', (tr_loss - logging_loss)/args['logging_steps'], global_step)
          logging_loss = tr_loss

        if args['save_steps'] > 0 and global_step % args['save_steps'] == 0:
          output_dir = os.path.join(args['output_dir'], 'checkpoint-{}'.format(global_step))
          if not os.path.exists(output_dir):
            os.makedirs(output_dir)
          model_to_save = model.module if hasattr(model, 'module') else model
          model_to_save.save_pretrained(output_dir)
          logger.info("Saving model checkpoint to %s", output_dir)

  return global_step, tr_loss / global_step

In [32]:
config_class, model_class, tokenizer_class = MODEL_CLASSES[args["model_type"]]
config = config_class.from_pretrained(args["model_name"], num_labels=2, finetuning_task="binary")
tokenizer = tokenizer_class.from_pretrained(args["model_name"])
model = model_class(config)
model.to(device)

train_dataset = generate_training_dataset_from_articles([train_articles], [train_SI_labels], tokenizer)
global_step, tr_loss = train(train_dataset, model, tokenizer)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)
logger.info("Saving model checkpoint to %s", args['output_dir'])
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(args['output_dir'])
tokenizer.save_pretrained(args['output_dir'])
torch.save(args, os.path.join(args['output_dir'], 'training_args.bin'))

INFO:LOG:Generating training dataset...
INFO:LOG:Generating dataframe for folder C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/train
INFO:LOG:Creating features from dataframe


      label                                               text
0         0  буковинця засудили до довічного ув'язнення за ...
1         1  чоловік побив її, потім наніс ножові поранення...
2         0                                                 о.
3         0                                     друзі, у мене 
4         1                                              шален
...     ...                                                ...
9902      1                   швидкість вітру понад 130 км/год
9903      0  . тим часом крим накрив найпотужніший шторм, я...
9904      0                                                   
9905      1                повальна мобілізація теж не допомож
9906      0  е. навіть якщо призвемо зараз 4 млн українців ...

[9907 rows x 2 columns]
(9907, 2)


INFO:LOG:Creating TensorDataset from features dataframe
C:\Users\Ната\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch:   0%|          | 0/1 [00:00<?, ?it/s]C:\Windows\Temp\ipykernel_3216\1346356067.py:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")


Iteration:   0%|          | 0/1239 [00:00<?, ?it/s]

Epoch: 100%|██████████| 1/1 [12:52<00:00, 772.62s/it]
INFO:LOG: global_step = 1239, average loss = 0.301520622600433
INFO:LOG:Saving model checkpoint to datasets/upsampling/output_model_SI


In [33]:
def generate_SI_eval_dataset_from_article(article_file, tokenizer):
  """
  Generates SI dataset to go into BERT from articles and labels
  """
  # If generating dataset for evaluation, do:
  logger.info("Generating evaluation dataset...")

  # Get the id name
  article_id = os.path.basename(article_file).split(".")[0][7:]

  # Read in the article
  with codecs.open(article_file, "r", encoding="utf8") as f:
    article = f.read()

  # For evaluating, do:
  dataframe = article_to_sequences(article_id, article, tokenizer)

  # Only include sequences that has alphanumeric characters in them
  bool_indices = [False] * len(dataframe)
  for i in range(len(dataframe)):
    if any(c.isalnum() for c in dataframe["text"][i]):
      bool_indices[i] = True
  dataframe = dataframe.loc[bool_indices].reset_index(drop=True)

  print(dataframe)
  print(dataframe.shape)

  # Process into features dataframe
  logger.info("Creating features from dataframe")
  features = convert_dataframe_to_features(dataframe, args['max_seq_length'], tokenizer)
      
  # Creating TensorDataset from features
  logger.info("Creating TensorDataset from features dataframe")
  all_input_ids = torch.tensor(features["input_ids"], dtype=torch.long)
  all_input_mask = torch.tensor(features["input_mask"], dtype=torch.long)
  all_segment_ids = torch.tensor(features["segment_ids"], dtype=torch.long)
  all_label_ids = torch.tensor(features["label_ids"], dtype=torch.long)

  dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
  return dataset, dataframe


In [34]:
def classify_per_article(eval_dataframe, eval_dataset, model, tokenizer):
  """
  Classifies a single article dataset and returns article id with indices list
  """
  # Load the eval data and initialize sampler
  eval_sampler = SequentialSampler(eval_dataset)
  eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

  # Start Classification
  #preds = None
  preds = np.empty((0, 2)) #!!!!

  # For each batch, evaluate
  for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2],
                'labels':         batch[3]}
      outputs = model(**inputs)
      logits = outputs[1]

      # Get predictions
    # if preds is None:
    #   
    #   preds = logits.detach().cpu().numpy()
    # else:

      preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)

  # Get the most probable prediction
  preds = np.argmax(preds, axis=1)
  eval_dataframe["label"] = preds
  print(eval_dataframe)
  # Get the propaganda indices and return them
  eval_dataframe = eval_dataframe.loc[eval_dataframe["label"] ==1].reset_index(drop=True)
  indices_list = []
  for i in range(len(eval_dataframe)):
    indices_list.append(tuple([eval_dataframe["seq_starts"][i], eval_dataframe["seq_ends"][i]]))
  indices_list = merge_overlapping(indices_list)

  return eval_dataframe["id"][0], indices_list

In [35]:
# As in Tutorial
articles_folder = dev_articles
article_filenames = sorted(glob.glob(os.path.join(articles_folder, "*.txt"))[128:])   # [59:100][100:140]128

labels_dir = 'C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/pred_SI_labels_upsampling'
if not os.path.exists(labels_dir):
  os.makedirs(labels_dir)

for i in range(len(article_filenames)):
  eval_dataset, eval_dataframe = generate_SI_eval_dataset_from_article(article_filenames[i], tokenizer)
  article_id, indices_list = classify_per_article(eval_dataframe, eval_dataset, model, tokenizer)
  
  f = open(labels_dir + '/article{}.labels.tsv'.format(article_id), 'w')
  writer = csv.writer(f, delimiter='\t')
  for indices in indices_list:
    writer.writerow([article_id, indices[0], indices[1]])
  f.close()
  


INFO:LOG:Generating evaluation dataset...
INFO:LOG:Creating features from dataframe


             id  seq_starts  seq_ends  label          text
0       1636698           0         2      0           " о
1       1636698           0         3      0          " ор
2       1636698           0         6      0       " орбан
3       1636698           0         7      0     " орбан "
4       1636698           0        10      0  " орбан " із
...         ...         ...       ...    ...           ...
224040  1636698        1895      1897      0            ви
224041  1636698        1895      1901      0        вибори
224042  1636698        1895      1902      0      вибори .
224043  1636698        1897      1901      0        ##бори
224044  1636698        1897      1902      0      ##бори .

[224045 rows x 5 columns]
(224045, 5)


INFO:LOG:Creating TensorDataset from features dataframe
C:\Windows\Temp\ipykernel_3216\1158175120.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):


Evaluating:   0%|          | 0/28006 [00:00<?, ?it/s]

INFO:LOG:Generating evaluation dataset...
INFO:LOG:Creating features from dataframe


...

INFO:LOG:Generating evaluation dataset...
INFO:LOG:Creating features from dataframe


          id  seq_starts  seq_ends  label                    text
0     995781          -1         4      0                   віина
1     995781          -1        10      1              віина може
2     995781          -1        16      1        віина може знову
3     995781          -1        19      1     віина може знову ви
4     995781          -1         2      1  віина може знову виити
...      ...         ...       ...    ...                     ...
8499  995781         429       435      1                ##писати
8500  995781         429       437      1              ##писатись
8501  995781         432       435      1                   ##ати
8502  995781         432       437      1                 ##атись
8503  995781         435       437      1                    ##сь

[8504 rows x 5 columns]
          id  seq_starts  seq_ends  label               text
0     996072           0         1      0                  у
1     996072           0         4      0               угор


INFO:LOG:Creating TensorDataset from features dataframe
C:\Windows\Temp\ipykernel_3216\1158175120.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):


Evaluating:   0%|          | 0/804 [00:00<?, ?it/s]

          id  seq_starts  seq_ends  label               text
0     996072           0         1      0                  у
1     996072           0         4      1               угор
2     996072           0         8      1           угорщина
3     996072           0        11      1        угорщина та
4     996072           0        17      1  угорщина та слова
...      ...         ...       ...    ...                ...
6425  996072         351       357      1           ##писати
6426  996072         351       359      1         ##писатись
6427  996072         354       357      1              ##ати
6428  996072         354       359      1            ##атись
6429  996072         357       359      1               ##сь

[6430 rows x 5 columns]


In [36]:
#Write in one file

f_total = open("C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/SI.labels_pred.txt", 'w')
writer = csv.writer(f_total, delimiter='\t')
labels_filenames = sorted(glob.glob(os.path.join(labels_dir, "*.labels.txt")))
for i in range(len(labels_filenames)):
  f = open(labels_filenames[i], 'r')
  reader = csv.reader(f, delimiter='\t')
  for row in reader:
    writer.writerow(row)
  f.close()
f_total.close()

In [5]:
# Open the output file in write mode
#!!!!!!!!!!!!!!!!!!!
labels_dir = 'C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/pred_SI_labels_upsampling'
with open("C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/SI.labels_pred.txt", 'w', newline='') as f_total:
    writer = csv.writer(f_total, delimiter='\t')
    print(f_total)
    # Get a sorted list of .txt files in the specified directory
    labels_filenames = sorted(glob.glob(os.path.join(labels_dir, "*.labels.tsv")))
    print(labels_filenames)
    # Iterate through each file and write its content to the output file
    for filename in labels_filenames:
        with open(filename, 'r') as f:
            reader = csv.reader(f, delimiter='\t')
            for row in reader:
                writer.writerow(row)

<_io.TextIOWrapper name='C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/SI.labels_pred.txt' mode='w' encoding='cp1251'>
['C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/pred_SI_labels_upsampling\\article1062476.labels.tsv', 'C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/pred_SI_labels_upsampling\\article1062542.labels.tsv', 'C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/pred_SI_labels_upsampling\\article1062916.labels.tsv', 'C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/pred_SI_labels_upsampling\\article1063082.labels.tsv', 'C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/pred_SI_labels_upsampling\\article1063158.labels.tsv', 'C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/pred_SI_labels_upsampling\\article1063177.labels.tsv', 'C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_

## Calculate accuracy

In [6]:
# Read contents of template.labels_pred.txt
pred_file = "C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/SI.labels_pred.txt"
true_file = "C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/upsampling/SI.labels_true.txt"

In [8]:
# Initialize dictionaries to store IDs and techniques
pred_dict = {}
true_dict = {}

# Read contents of TC.labels_pred.txt
with open(pred_file, "r") as pred_f:
    pred_reader = csv.reader(pred_f, delimiter="\t")
    for row in pred_reader:
      if len(row)>=3:
        id_ = row[0]
        start = int(row[1])
        end = int(row[2])
        if id_ not in pred_dict:
            pred_dict[id_] = [(start, end)]
        else:
            pred_dict[id_].append((start, end))
      # else:
      # # Handle cases where the row doesn't have enough elements
      #   print("Row does not have enough elements:", row)
print (len(pred_dict)) 

#Read contents of TC.labels_true.txt
with open(true_file, "r") as true_f:
    true_reader = csv.reader(true_f, delimiter="\t")
    for row in true_reader:
        id_ = row[0]
        start = int(row[1])
        end = int(row[2])
        if id_ not in true_dict:
            true_dict[id_] = [(start, end)]
        else:
            true_dict[id_].append((start, end))
print (len(true_dict)) 

595
595


In [12]:
# Convert -1 to 0
for key, value in pred_dict.items():
    # Iterate through each tuple in the list
    for i, (start, end) in enumerate(value):
        # Check if the start value is -1
        if start == -1:
            # Replace -1 with 0
            pred_dict[key][i] = (0, end)

print(pred_dict)

{'1062476': [(0, 290)], '1062542': [(1, 39), (41, 53), (54, 62), (63, 85), (86, 100), (101, 109), (110, 134), (135, 149), (150, 158), (159, 182), (183, 251), (254, 283)], '1062916': [(0, 582)], '1063082': [(0, 105), (107, 204), (206, 248)], '1063158': [(0, 419)], '1063177': [(0, 249)], '1063217': [(0, 338)], '1063308': [(0, 391)], '1063340': [(0, 599)], '1063449': [(0, 259)], '1063475': [(0, 336)], '1064347': [(0, 204)], '1064382': [(0, 324)], '1064725': [(0, 102), (103, 184), (186, 211), (213, 227), (229, 287)], '1064779': [(0, 244)], '1065168': [(0, 308)], '1065870': [(0, 328), (330, 351), (354, 384)], '1066039': [(0, 342)], '1066310': [(0, 314)], '1066612': [(0, 345)], '1066986': [(0, 268), (271, 299)], '1067012': [(0, 340)], '1067320': [(0, 215), (217, 251)], '1067379': [(0, 286)], '1067429': [(0, 235), (237, 265)], '1067452': [(0, 155), (157, 215), (217, 242), (245, 273)], '1067800': [(0, 598)], '1068004': [(0, 344)], '1068121': [(0, 252), (255, 283)], '1068417': [(0, 220), (222, 

In [13]:
pred_dict

{'1062476': [(0, 290)],
 '1062542': [(1, 39),
  (41, 53),
  (54, 62),
  (63, 85),
  (86, 100),
  (101, 109),
  (110, 134),
  (135, 149),
  (150, 158),
  (159, 182),
  (183, 251),
  (254, 283)],
 '1062916': [(0, 582)],
 '1063082': [(0, 105), (107, 204), (206, 248)],
 '1063158': [(0, 419)],
 '1063177': [(0, 249)],
 '1063217': [(0, 338)],
 '1063308': [(0, 391)],
 '1063340': [(0, 599)],
 '1063449': [(0, 259)],
 '1063475': [(0, 336)],
 '1064347': [(0, 204)],
 '1064382': [(0, 324)],
 '1064725': [(0, 102), (103, 184), (186, 211), (213, 227), (229, 287)],
 '1064779': [(0, 244)],
 '1065168': [(0, 308)],
 '1065870': [(0, 328), (330, 351), (354, 384)],
 '1066039': [(0, 342)],
 '1066310': [(0, 314)],
 '1066612': [(0, 345)],
 '1066986': [(0, 268), (271, 299)],
 '1067012': [(0, 340)],
 '1067320': [(0, 215), (217, 251)],
 '1067379': [(0, 286)],
 '1067429': [(0, 235), (237, 265)],
 '1067452': [(0, 155), (157, 215), (217, 242), (245, 273)],
 '1067800': [(0, 598)],
 '1068004': [(0, 344)],
 '1068121': [(

In [14]:
# Calculate the total number of spans in predicted dictionary
total_spans = sum(len(spans) for spans in pred_dict.values())
# Calculate the average number of spans per key
average_spans_per_key = total_spans / len(pred_dict)
print(f"Average number of spans per key: {average_spans_per_key:.2f}")

Average number of spans per key: 1.27


In [15]:
# Calculate the total length between end and start for all spans
total_length = sum(end - start for spans in pred_dict.values() for start, end in spans)
# Calculate the total number of spans
total_spans = sum(len(spans) for spans in pred_dict.values())
# Calculate the average length
average_length = total_length / total_spans
print(f"Average length of the span: {average_length:.2f}")

Average length of the span: 198.39


In [11]:
true_dict

{'20120': [(84, 226)],
 '20220': [(0, 197)],
 '20290': [(0, 203)],
 '20330': [(0, 205)],
 '20360': [(0, 241)],
 '20420': [(0, 269)],
 '20460': [(0, 218)],
 '20470': [(0, 229)],
 '20500': [(0, 227)],
 '20520': [(0, 233)],
 '20600': [(0, 200)],
 '20650': [(0, 222)],
 '20710': [(0, 182)],
 '20770': [(0, 189)],
 '20810': [(0, 232)],
 '20880': [(0, 228)],
 '20990': [(0, 261)],
 '21060': [(0, 87)],
 '21100': [(0, 86)],
 '21140': [(0, 85)],
 '21220': [(0, 66)],
 '21250': [(0, 86)],
 '21260': [(0, 54)],
 '21290': [(0, 97)],
 '21360': [(0, 70)],
 '21520': [(0, 79)],
 '21610': [(0, 77)],
 '21640': [(0, 92)],
 '21660': [(0, 86)],
 '21760': [(0, 265)],
 '21770': [(0, 135)],
 '21840': [(0, 198)],
 '21870': [(0, 138)],
 '21900': [(0, 113)],
 '21930': [(0, 116)],
 '22040': [(0, 85)],
 '22140': [(0, 134)],
 '22230': [(0, 108)],
 '22370': [(0, 73)],
 '22410': [(0, 82)],
 '22430': [(0, 100)],
 '22690': [(0, 134)],
 '22700': [(0, 105)],
 '22710': [(0, 142)],
 '22750': [(0, 153)],
 '22920': [(0, 150)],
 '

In [16]:
# Calculate the total number of spans in true dictionary
total_spans = sum(len(spans) for spans in true_dict.values())
# Calculate the average number of spans per key
average_spans_per_key = total_spans / len(true_dict)
print(f"Average number of spans per key: {average_spans_per_key:.2f}")

Average number of spans per key: 1.42


In [17]:
# Calculate the total length between end and start for all spans
total_length = sum(end - start for spans in true_dict.values() for start, end in spans)
# Calculate the total number of spans
total_spans = sum(len(spans) for spans in true_dict.values())
# Calculate the average length
average_length = total_length / total_spans
print(f"Average length of the span: {average_length:.2f}")

Average length of the span: 61.29


In [18]:
# Standart overlap
overlaps = []

# Iterate through each ID in the true dictionary
for id, true_spans in true_dict.items():
    
    # Check if the ID is present in the predicted dictionary
    if id in pred_dict:
        
        # Get the predicted spans for the current ID
        pred_spans = pred_dict[id]
        #print(pred_spans)
        
        true_length = 0
        pred_length = 0
    
#         # Iterate through each true span
        for true_span in true_spans:
            max_overlap = 0
#             
#             # Extract start and end positions of the true span
            true_start, true_end = true_span

            
#             # Iterate through each predicted span
            for pred_span in pred_spans:
                # Extract start and end positions of the predicted span
                pred_start, pred_end = pred_span
                  
#                 # Calculate the overlap between the true and predicted spans
                overlap = max(0, min(true_end, pred_end) - max(true_start, pred_start))
                #print(pred_span)
                #print(true_span)
                #print(overlap)
#                 # Update the maximum overlap if necessary
                #max_overlap = max(max_overlap, overlap)

#             # Append the maximum overlap for the current true span to the list
            overlaps.append(overlap)


#print(overlaps)        
# print(sum(overlaps))
# # Calculate the accuracy
average_overlap = sum(overlaps) / len(overlaps) if overlaps else 0
# precision = 
# 
print(f"Accuracy on character level: {average_overlap:.2f}")

Accuracy on character level: 56.66


#### Analyze syntatic dataset vs initial

In [40]:
# Analyze syntatic dataset vs initial 
syntatic_data = pd.read_csv("C:/N\st/UCU/5sem2024_DIPLOMA/Manipulation/code/labels/generated_texts/generated_texts.csv")
syntatic_ids = syntatic_data[["message_id"]]
syntatic_ids = syntatic_ids["message_id"].tolist()
syntatic_ids = [str(key) for key in syntatic_ids]
syntatic_ids

['60010',
 '60020',
 '60030',
 '60040',
 '60050',
 '60060',
 '60070',
 '60080',
 '60090',
 '60100',
 '60110',
 '60120',
 '60130',
 '60140',
 '60150',
 '60160',
 '60170',
 '60180',
 '60190',
 '60200',
 '60210',
 '60220',
 '60230',
 '60240',
 '60250',
 '60260',
 '60270',
 '60280',
 '60290',
 '60300',
 '60310',
 '60320',
 '60330',
 '60340',
 '60350',
 '60360',
 '60370',
 '60380',
 '60390',
 '60400',
 '60410',
 '60420',
 '60430',
 '60440',
 '60450',
 '60460',
 '60470',
 '60480',
 '60490',
 '60500',
 '60510',
 '60520',
 '60530',
 '60540',
 '60550',
 '60560',
 '60570',
 '60580',
 '60590',
 '60600',
 '60610',
 '60620',
 '60630',
 '60640',
 '60650',
 '60660',
 '60670',
 '60680',
 '60690',
 '60700',
 '60710',
 '60720',
 '60730',
 '60740',
 '60750',
 '60760',
 '60770',
 '60780',
 '60790',
 '60800',
 '60810',
 '60820',
 '60830',
 '60840',
 '60850',
 '60860',
 '60870',
 '60880',
 '60890',
 '60900',
 '60910',
 '60920',
 '60930',
 '60940',
 '60950',
 '60960',
 '60970',
 '60980',
 '60990',
 '61000',


In [43]:
test = {'21670': [(0, 77)],
 '21680': [(0, 92)],
 '21690': [(0, 86)],
 '21760': [(0, 265)],
 '21770': [(0, 135)],
 '21840': [(0, 198)],
 '21870': [(0, 138)],
 '21900': [(0, 113)],
 '21930': [(0, 116)],
 '22040': [(0, 85)]
    
}


In [47]:
# Create a pred dictionary with only the keys that are in the list with synthetic data
pred_dict_syn = {k: v for k, v in pred_dict.items() if k in syntatic_ids}
# filtered_dict = {}
# for ids,k in test.items():
#     # if not isinstance(ids, str):
#     #     ids = str(ids)
#     if ids in syntatic_ids:
#         filtered_dict[ids] = k
# print(filtered_dict)
len(pred_dict_syn)

317

In [53]:
# Standart overlap only with syntatic data
overlaps = []

# Iterate through each ID in the true dictionary
for id, true_spans in true_dict.items():
    
    # Check if the ID is present in the predicted dictionary
    if id in pred_dict_syn:
        
        # Get the predicted spans for the current ID
        pred_spans = pred_dict_syn[id]
        #print(pred_spans)
        
        true_length = 0
        pred_length = 0
    
#         # Iterate through each true span
        for true_span in true_spans:
            max_overlap = 0
#             
#             # Extract start and end positions of the true span
            true_start, true_end = true_span
            true_span_length = true_end - true_start
            
#             # Iterate through each predicted span
            for pred_span in pred_spans:
                # Extract start and end positions of the predicted span
                pred_start, pred_end = pred_span
                  
#                 # Calculate the overlap between the true and predicted spans
                overlap = max(0, min(true_end, pred_end) - max(true_start, pred_start))
                #print(pred_span)
                #print(true_span)
                #print(overlap)
#                 # Update the maximum overlap if necessary
                max_overlap = max(max_overlap, overlap)

#             # Append the maximum overlap for the current true span to the list
            #overlaps.append(overlap)
            if true_span_length > 0:
                normalized_overlap = (max_overlap / true_span_length) * 100
                overlaps.append(normalized_overlap)


#print(overlaps)        
# print(sum(overlaps))
# # Calculate the accuracy
average_overlap = sum(overlaps) / len(overlaps) if overlaps else 0

print(f"Accuracy on character level: {average_overlap:.2f}")
print(sum(overlaps))
print(len(overlaps))

Accuracy on character level: 99.65
31590.081135808952
317


In [54]:
# Create a pred dictionary with only the keys that are in the list with synthetic data
pred_dict_not_syn = {k: v for k, v in pred_dict.items() if k not in syntatic_ids}
len(pred_dict_not_syn)

278

In [55]:
# Standart overlap only with syntatic data
overlaps = []

# Iterate through each ID in the true dictionary
for id, true_spans in true_dict.items():
    
    # Check if the ID is present in the predicted dictionary
    if id in pred_dict_not_syn:
        
        # Get the predicted spans for the current ID
        pred_spans = pred_dict_not_syn[id]
        #print(pred_spans)
        
        true_length = 0
        pred_length = 0
    
#         # Iterate through each true span
        for true_span in true_spans:
            max_overlap = 0
#             
#             # Extract start and end positions of the true span
            true_start, true_end = true_span
            true_span_length = true_end - true_start
            
#             # Iterate through each predicted span
            for pred_span in pred_spans:
                # Extract start and end positions of the predicted span
                pred_start, pred_end = pred_span
                  
#                 # Calculate the overlap between the true and predicted spans
                overlap = max(0, min(true_end, pred_end) - max(true_start, pred_start))
                #print(pred_span)
                #print(true_span)
                #print(overlap)
#                 # Update the maximum overlap if necessary
                max_overlap = max(max_overlap, overlap)

#             # Append the maximum overlap for the current true span to the list
            #overlaps.append(overlap)
            if true_span_length > 0:
                normalized_overlap = (max_overlap / true_span_length) * 100
                overlaps.append(normalized_overlap)


#print(overlaps)        
# print(sum(overlaps))
# # Calculate the accuracy
average_overlap = sum(overlaps) / len(overlaps) if overlaps else 0

print(f"Accuracy on character level: {average_overlap:.2f}")
print(sum(overlaps))
print(len(overlaps))

Accuracy on character level: 99.31
52136.5527055113
525


Ref: https://medium.com/@jihwangk/fine-grained-propaganda-detection-and-classification-with-bert-dfad4acaa321
- https://github.com/ThilinaRajapakse/pytorch-transformers-classification/blob/master/colab_quickstart.ipynb